# IMPORTS

In [1]:
import pandas as pd

import my_pickle as mp
import my_split as ms
import my_dataframes as md

from importlib import reload

# READ IN

In [2]:
# read in message data
message_df = md.get_message_df()
convo_df = md.get_conversation_df(message_df)
last_df = md.get_lastmessage_df()

created message dataframe
created conversation dataframe with 0 known errors
created last message dataframe


In [3]:
convo_df.head()

,conv_id,response,first_uid,second_uid,first_mid,second_mid,timestamp
16296,rKdn0zblFiRVDUEmDHyQ,True,rKdn0zblFi,RVDUEmDHyQ,-KWKghvlpF67UOv-iXXO,-KWKg_ybWV4qbG02NYrb,1478906535730
9241,aXnWmLjpaERVDUEmDHyQ,False,aXnWmLjpaE,RVDUEmDHyQ,-KWLsSEmEJgH3bhJupE7,None,1478926390334
1069,8Ws3QWmC4B52lmw7KQcq,True,8Ws3QWmC4B,52lmw7KQcq,-KWPb8hd_DbLpY1eIUlT,-KWQNe2ghZCGV0aywnKp,1478988962771
9414,b1HYtkC11e92RR2gpmjr,False,b1HYtkC11e,92RR2gpmjr,-KWQMn5WmLIKbUOt9a0O,None,1479001715202
4142,KnmYDsn5Q492RR2gpmjr,False,KnmYDsn5Q4,92RR2gpmjr,-KWQNFIghvLurnI-8epo,None,1479001834795


# ECT SPLIT

In [4]:
# find the split!
cutoff = ms.ect_find_split(convo_df, .7)
print("split on:         {}".format(pd.to_datetime(cutoff*1000000)))

# do the split!
safe_convo_df = ms.ect_make_split(convo_df, cutoff)
print("most recent data: {}".format(pd.to_datetime(safe_convo_df.timestamp.max()*1000000)))

split on:         2017-10-02 01:35:58.644000
most recent data: 2017-10-02 01:35:58.644000


# COMBINE USER AND MESSAGE FILES

NB: Have to do after ECT split!

In [5]:
# get a data table with messages sent/received for each user
response_df = md.get_response_df(convo_df)
response_df.head(2)

,messages_sent,responses_received,attractiveness,messages_received,responses_sent,selectivity
00DMWuKDFs,0.0,0.0,0.392391,17.0,0.0,1.0
00kOFrnYJu,0.0,0.0,0.392391,2.0,0.0,1.0


In [6]:
# Combine with user table
user_df = md.get_user_data(response_df)    # currently from csv, will be from json

user_df.head(2)

,created,updated,about,has_about,len_about,available,birthday,age,collegeId,emailVerified,...,roomTypeId,smokingOk,term,work,messages_sent,responses_received,attractiveness,messages_received,responses_sent,selectivity
uid,,,,,,,,,,,,,,,,,,,,,
0a9yOPKFSH,2017-06-19 03:31:08,2018-01-04 21:24:41,,False,0,nan,1992-01-01,26.0,NaN,1,...,2.0,0.0,12.0,SEG,7.0,2.0,0.285714,9.0,2.0,0.777778
0owxBbcyZ6,2017-10-12 00:32:29,2018-01-04 03:05:50,,False,0,nan,1974-01-01,44.0,NaN,1,...,NaN,1.0,NaN,WA State,0.0,0.0,0.392391,7.0,0.0,1.000000


# REMOVE ROWS FROM CONVOS WITH MISSING UIDS

In [8]:
safe_convo_df = md.remove_bad_uids(safe_convo_df, user_df)  

/Users/gandalf/Documents/Galvanize/MatchingService/my_dataframes.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['flag1'] = df.first_uid.apply(lambda x: x not in uids)
/Users/gandalf/Documents/Galvanize/MatchingService/my_dataframes.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['flag2'] = df.second_uid.apply(lambda x: x not in uids)
/Users/gandalf/Documents/Galvanize/MatchingService/my_dataframes.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [9]:
safe_convo_df.head()

,conv_id,response,first_uid,second_uid,first_mid,second_mid,timestamp
14397,nQYG2Z9elg8Ws3QWmC4B,True,nQYG2Z9elg,8Ws3QWmC4B,-KWZ6j5vHnNWPD90JJ18,-KWUqwqvF7RfcRaEqtL6,1479148499453
5891,QkEYa5ZJNZ6Iol6V4bND,True,6Iol6V4bND,QkEYa5ZJNZ,-KW_4Mfb_a0bNlSYtwN0,-KW_4ID8hJs4Hw_lU81P,1479164656685
14184,n2ffQeQMriYL4dlYdM1W,True,YL4dlYdM1W,n2ffQeQMri,-KYLE9PvLEsrBHm36puZ,-KYLDu4jBBzdhx8VdHcD,1481063049035
8589,YL4dlYdM1W6Iol6V4bND,True,YL4dlYdM1W,6Iol6V4bND,-KYLEFnJYvUAIbEW4uvC,-KYLPbedA40Fz69oKze4,1481063075142
14404,nQYG2Z9elgn2ffQeQMri,False,n2ffQeQMri,nQYG2Z9elg,-KYLEyp9_LdGKEMioVyZ,None,1481063263841


# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [10]:
mp.pickle_it(user_df,'data_user')
# mp.pickle_it(message_df,'message_df')  NOT ECT SAFE!
mp.pickle_it(safe_convo_df,'data_convo')  # make sure pickling safe stuff
# pickle_it(lastmessage_df,'lastmessage_df') NOT ECT SAFE!